[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Denis-R-V/TSR/blob/main/notebooks/6.2.inference_bot.ipynb)

# Система распознавания дорожных знаков на датасете RTSD

## Детектор. Инференс (Telegram Bot)

In [1]:
# если работаем в колабе - монтируем диск
try:
    from google.colab import drive
    drive.mount('/content/drive')  
    colab=True
except:
    colab=False

In [2]:
#import json
import os
import re
import sys

#import matplotlib.patches as patches
#import matplotlib.pyplot as plt
import telebot
import torch
#import torch.nn as nn
#import torch.utils.data
#import torchvision
#import torchvision.transforms as transforms
#from PIL import Image, ImageDraw, ImageFont
#from torchvision.models import resnet152
#from torchvision.models.detection import FasterRCNN
#from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

sys.path.append('../')
from config import token
from src.execute import Builder

#%matplotlib inline

### Пути и параметры

In [3]:
device_id = 0
device = f'cuda:{device_id}' if torch.cuda.is_available() else 'cpu'

#dataset_path = 'data/raw/RTSD' if colab else os.path.join('..', 'data', 'raw', 'RTSD')
data_prepared_path = '../content/drive/MyDrive/TSR/data/prepared' if colab else os.path.join('..', 'data', 'prepared')
models_path = '../content/drive/MyDrive/TSR/models' if colab else os.path.join('..', 'models')
images_path = '../content/drive/MyDrive/TSR/images/telebot_images' if colab else os.path.join('..', 'images', 'telebot_images')
if not os.path.exists(images_path): os.makedirs(images_path)

detector_file = 'chkpt_detector_resnet50_v2_augmented_b8_5.pth'
classifier_file = 'classifier_resnet152_add_signs_bg100_tvs_randomchoice_perspective_colorjitter_resizedcrop_erasing_adam_001_sh_10_06_model_29.pth'

detector_threshold = 0.9
classifier_threshold = 0.9
debug_mode = False

### Загрузка модели (детектор и классификатор) и параметров

In [4]:
model = Builder(device=device,
                detector_path=os.path.join(models_path, detector_file),
                classifier_path=os.path.join(models_path, classifier_file),
                detector_threshold=detector_threshold,
                classifier_threshold=classifier_threshold,
                debug_mode=debug_mode)

Добавить файлы со знаками
Для FasterRCNN с backbone resnet50v2 загружены веса из ..\models\chkpt_detector_resnet50_v2_augmented_b8_5.pth
Загружен классификатор из ..\models\classifier_resnet152_add_signs_bg100_tvs_randomchoice_perspective_colorjitter_resizedcrop_erasing_adam_001_sh_10_06_model_29.pth


### Telegram bot

In [5]:
'''        new_image = img_test.copy()
        font = ImageFont.load_default()
        #font = ImageFont.truetype('arial.ttf', size=18)
        pencil = ImageDraw.Draw(new_image)
        for i in range(len((result[0]))):
            pencil.rectangle(result[0][i], fill = None, width=2, outline='yellow')
            text_x = result[0][i][0]
            text_y = result[0][i][1]
            mark = str(result[1][i]) + ': ' + str(round(result[2][i], 2))
            pencil.text((text_x, text_y - 9), mark, font=font, fill = 'red', size = 20)'''

"        new_image = img_test.copy()\n        font = ImageFont.load_default()\n        #font = ImageFont.truetype('arial.ttf', size=18)\n        pencil = ImageDraw.Draw(new_image)\n        for i in range(len((result[0]))):\n            pencil.rectangle(result[0][i], fill = None, width=2, outline='yellow')\n            text_x = result[0][i][0]\n            text_y = result[0][i][1]\n            mark = str(result[1][i]) + ': ' + str(round(result[2][i], 2))\n            pencil.text((text_x, text_y - 9), mark, font=font, fill = 'red', size = 20)"

In [6]:
bot = telebot.TeleBot(token)

@bot.message_handler(content_types=['text', 'photo'])
def get_text_message(message):
    if message.text:                                    # to do если в тексте есть классификатор и детектор - ставим оба, если один - по одному, если не одного - запрос 1 или 2 (оба 3)
        message_text = message.text
        if re.search(r'0[\.,\s]\d+', message.text):
            digit_board = re.search('0[\.,\s]\d+', message.text).span()
            temp = digit_board
            threshold = message.text[digit_board[0] : digit_board[1]]
            threshold = re.sub(r',|\s', '.', threshold)
            threshold = round(float(threshold), 2)
            bot.send_message(message.from_user.id, f'Установлен threshold = {model.detector_threshold}')
        
        elif re.search('трешхолд|трэшхолд|threshold', message.text.lower()):
            bot.send_message(message.from_user.id, f'Threshold = {model.detector_threshold}')

        elif re.search('debug|дебаг', message.text.lower()):
            model.debug_mode = not model.debug_mode
            bot.send_message(message.from_user.id, f'Установлен debug_mode = {model.debug_mode}')
        
        else:
            bot.send_message(message.from_user.id, 'Привет!\nБот принимает фотографию и возвращает фотографию с отмеченными дорожными знаками и их названиями.')
    
    elif message.photo:
        #image = message.photo
        #bot.send_message(message.from_user.id, 'Получено фото')
        #file_path = '.'
        raw = message.photo[3].file_id
        name = raw + '.jpg'
        file_info = bot.get_file(raw)
        downloaded_file = bot.download_file(file_info.file_path)
        with open(os.path.join(images_path, name), 'wb') as new_file:
            new_file.write(downloaded_file)
        
        #img = Image.open(os.path.join(images_path, name), 'r')
        
        img_pred, description = model.predict_single_visualized(os.path.join(images_path, name))
        
        labels_names = '\n'.join(description)

        #bot.send_message(message.from_user.id, str(result))
        bot.send_photo(message.from_user.id, img_pred)
        bot.send_message(message.from_user.id, labels_names)
        
bot.polling(none_stop=True, interval=0)

In [73]:
bot = telebot.TeleBot(token)

@bot.message_handler(content_types=['text', 'photo'])
def get_text_message(message):
    if message.text:                                    # to do если в тексте есть классификатор и детектор - ставим оба, если один - по одному, если не одного - запрос 1 или 2 (оба 3)
        message_text = message.text
        if re.search(r'0[\.,\s]\d+', message.text):
            digit_board = re.search('0[\.,\s]\d+', message.text).span()
            temp = digit_board
            threshold = message.text[digit_board[0] : digit_board[1]]
            threshold = re.sub(r',|\s', '.', threshold)
            threshold = round(float(threshold), 2)
            bot.send_message(message.from_user.id, f'Установлен threshold = {model.detector_threshold}')
        
        elif re.search('трешхолд|трэшхолд|threshold', message.text.lower()):
            bot.send_message(message.from_user.id, f'Threshold = {model.detector_threshold}')

        elif re.search('debug|дебаг', message.text.lower()):
            model.debug_mode = not model.debug_mode
            bot.send_message(message.from_user.id, f'Установлен debug_mode = {model.debug_mode}')
        
        else:
            bot.send_message(message.from_user.id, 'Привет!\nБот принимает фотографию и возвращает фотографию с отмеченными дорожными знаками и их названиями.')
    
    elif message.photo:
        image = message.photo
        #bot.send_message(message.from_user.id, 'Получено фото')
        file_path = '.'
        raw = message.photo[3].file_id
        name = raw + '.jpg'
        file_info = bot.get_file(raw)
        downloaded_file = bot.download_file(file_info.file_path)
        with open(os.path.join(images_path, name), 'wb') as new_file:
            new_file.write(downloaded_file)
        img = Image.open(os.path.join(images_path, name), 'r')
        img_test = img
        
        result = model.predict_single(img)

        new_image = img_test.copy()
        font = ImageFont.load_default()
        #font = ImageFont.truetype('arial.ttf', size=18)
        pencil = ImageDraw.Draw(new_image)
        for i in range(len((result[0]))):
            pencil.rectangle(result[0][i], fill = None, width=2, outline='yellow')
            text_x = result[0][i][0]
            text_y = result[0][i][1]
            mark = str(result[1][i]) + ': ' + str(round(result[2][i], 2))
            pencil.text((text_x, text_y - 9), mark, font=font, fill = 'red', size = 20)
        
        with open(os.path.join(data_prepared_path, 'labels_names_map.json'), 'r') as read_file:
            labels_names_map = json.load(read_file)
        read_file.close()

        labels = []
        if result[1] != []:
            for res in result[1]:
                labels.append(res)
        labels = list(set(labels))
        labels.sort()
        labels_names = []
        for label in labels:
            labels_names.append(f"{label}: {labels_names_map.get(label)}")
        
        labels_names = '\n'.join(labels_names)

        #bot.send_message(message.from_user.id, str(result))
        bot.send_photo(message.from_user.id, new_image)
        bot.send_message(message.from_user.id, labels_names)
        
bot.polling(none_stop=True, interval=0)

<>:19: SyntaxWarning: invalid escape sequence '\.'
<>:19: SyntaxWarning: invalid escape sequence '\.'
/var/folders/kq/gcmv7zs911x1dpngq2dt4m_00000gn/T/ipykernel_60242/3232980988.py:19: SyntaxWarning: invalid escape sequence '\.'
  digit_board = re.search('0[\.,\s]\d+', message.text).span()
